In [1]:
import pandas as pd
import dash 
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import plotly.express as px

In [2]:
df = pd.read_csv(r'E:\Machine Learning\Module 9\Social-Capital-Project\social-capital-project.csv')

In [3]:
df.head()

,State,County,% adults graduated high school,% in fair or poor health,% without health insurance,bubble_size,Unemployment rate,Prime-age male labor force participation,Median household income,Poverty rate,...,Black-white segregation,% foreign born,Population,Density,% rural,Mean commute time,% of housing owner-occupied,median age housing,Net migration,Average temperature
0,Alabama,Autauga,87.6,17.9,8.9,30,3.4,86.8,53099.0,12.3,...,0.38,1.8,55049,92.6,42.0,5.3,73.2,27,0.0093,77.7
1,Alabama,Baldwin,90.0,16.4,11.8,30,3.7,89.3,51365.0,13.0,...,0.32,3.3,199510,125.5,42.3,8.3,71.7,24,0.0183,78.2
2,Alabama,Barbour,73.8,25.0,13.0,30,6.2,83.6,33956.0,26.4,...,0.27,2.9,26614,30.1,67.8,7.3,63.9,37,-0.0162,79.0
3,Alabama,Bibb,80.7,19.5,9.0,30,3.5,84.3,39776.0,16.5,...,0.47,1.4,22572,36.3,68.4,7.5,72.6,35,-0.0053,76.8
4,Alabama,Blount,80.0,18.9,11.2,30,3.0,78.1,46212.0,16.5,...,0.46,4.3,57704,89.5,90.0,17.1,78.8,31,-0.0047,73.8


In [4]:
df.tail()

,State,County,% adults graduated high school,% in fair or poor health,% without health insurance,bubble_size,Unemployment rate,Prime-age male labor force participation,Median household income,Poverty rate,...,Black-white segregation,% foreign born,Population,Density,% rural,Mean commute time,% of housing owner-occupied,median age housing,Net migration,Average temperature
3137,Wyoming,Sweetwater,90.7,15.6,13.2,30,3.7,94.5,68233.0,11.0,...,0.28,5.5,44812,4.3,10.9,6.2,72.5,40,-0.0224,50.2
3138,Wyoming,Teton,95.4,11.9,14.7,30,1.7,97.4,75594.0,7.3,...,0.41,11.5,22623,5.7,46.4,1.4,59.8,29,-0.0139,42.1
3139,Wyoming,Uinta,89.6,14.7,15.7,30,4.2,90.0,53323.0,16.8,...,0.34,4.0,20893,10.0,43.1,9.1,70.3,36,-0.0042,47.6
3140,Wyoming,Washakie,87.4,14.4,16.9,30,4.6,91.0,46212.0,14.7,...,0.18,3.8,8351,3.7,36.0,2.9,74.5,47,-0.0193,54.6
3141,Wyoming,Weston,91.4,13.5,14.3,30,2.1,94.0,55640.0,14.7,...,0.35,4.1,7175,3.0,54.5,22.8,76.8,43,-0.0152,57.6


In [ ]:
df['State'].unique()

In [5]:
app = dash.Dash(__name__)

app.layout= html.Div([
    html.Label('State Information', style= {'textAlign': 'center'}),

    dcc.Dropdown(
        id= 'States_name',
        options= [{'label': s, 'value': s} for s in sorted(df['State'].unique())],
        value= None,
        clearable= False
    ),
    html.Label('Country Information', style= {'textAlign': 'center'}),

    dcc.Dropdown(
        id= 'Country_name',
        options= [],
        value= [],
        multi= True
    ),

    html.Div(id= 'Graph_1', children= [])
])

@app.callback(
    Output(component_id= 'Country_name', component_property= 'options'),
    Output(component_id= 'Country_name', component_property= 'value'),
    [Input(component_id= 'States_name', component_property= 'value')]
)
def set_options(dropdown):
    df_1= df[df['State'] == dropdown]
    countries_state= [{'label': s, 'value': s} for s in sorted(df_1['County'].unique())]
    value_select= [x['value'] for x in countries_state]
    return countries_state,value_select

@app.callback(
    Output(component_id= 'Graph_1', component_property= 'children'),
    [Input(component_id= 'Country_name', component_property= 'value')],
    State(component_id= 'States_name', component_property= 'value'),
    prevent_initial_call= True
)
def update_grpah(selected_counties, selected_state):
    if len(selected_counties) == 0:
        pass
    else:
        dff = df[(df['State']==selected_state) & (df.County.isin(selected_counties))]

        fig = px.scatter(dff, x='% without health insurance', y='% in fair or poor health',
                         color='% adults graduated high school',
                         trendline='ols',
                         size='bubble_size',
                         hover_name='County',
                         hover_data={'bubble_size':False},
                         labels={'% adults graduated high school':'% graduated high school',
                                 '% without health insurance':'% no health insurance',
                                 '% in fair or poor health':'% poor health'}
                         )
        return dcc.Graph(id='display-map', figure=fig)
if __name__== '__main__':
    app.run(debug= True)

---------------------------------------------------------------------------
ModuleNotFoundError                       Traceback (most recent call last)
Cell In[5], line 47, in update_grpah(
    selected_counties=['Apache', 'Cochise', 'Coconino', 'Gila', 'Graham', 'Greenlee', 'La Paz', 'Maricopa', 'Mohave', 'Navajo', 'Pima', 'Pinal', 'Santa Cruz', 'Yavapai', 'Yuma'],
    selected_state='Arizona'
)
     44 else:
     45     dff = df[(df['State']==selected_state) & (df.County.isin(selected_counties))]
---> 47     fig = px.scatter(dff, x='% without health insurance', y='% in fair or poor health',
        dff =        State      County  % adults graduated high school  \
96   Arizona      Apache                            78.2   
97   Arizona     Cochise                            86.6   
98   Arizona    Coconino                            88.9   
99   Arizona        Gila                            84.3   
100  Arizona      Graham                            85.4   
101  Arizona    Greenlee  